In [ ]:
!pip install stop_words
!pip install pymorphy2

  Created wheel for stop-words: filename=stop_words-2018.7.23-cp37-none-any.whl size=32917 sha256=eae43a305bb969ad98520653efff3e9e2b81f6fc3986cf2bab9aecb07f965d70
  Stored in directory: /root/.cache/pip/wheels/75/37/6a/2b295e03bd07290f0da95c3adb9a74ba95fbc333aa8b0c7c78
Successfully built stop-words
     |████████████████████████████████| 61kB 3.2MB/s 
     |████████████████████████████████| 8.2MB 7.4MB/s 


In [ ]:
# попробуем запрограммировать простую рекурентную сеть. Возьмем датасет с прошлого занятия

import pandas as pd
from string import punctuation
from stop_words import get_stop_words
from pymorphy2 import MorphAnalyzer
import re



In [ ]:
import pandas as pd

data = pd.read_csv('data/twe.csv')
data.head()
X = data['tweet']
y = data['label']

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.1, random_state=42)

In [ ]:
df_train_dict = {'text':X_train.values, 'class':y_train.values}
df_train = pd.DataFrame(df_train_dict)
df_test_dict = {'text':X_test.values, 'class':y_test.values}
df_test = pd.DataFrame(df_test_dict)
df_val_dict = {'text':X_val.values, 'class':y_val.values}
df_val = pd.DataFrame(df_val_dict)

In [ ]:
df_train.head()

,text,class
0,2night is the season finale of #livandmaddie o...,0
1,my day is officially ruined! builders have com...,0
2,"i have my bed, food and netflix",0
3,after an hour of walking and looking for somet...,0
4,@user im not gonna lie i did think angela was...,0


In [ ]:
sw = set(get_stop_words("ru"))
exclude = set(punctuation)
morpher = MorphAnalyzer()

def preprocess_text(txt):
    txt = str(txt)
    txt = "".join(c for c in txt if c not in exclude)
    txt = txt.lower()
    txt = re.sub("\sне", "не", txt)
    txt = [morpher.parse(word)[0].normal_form for word in txt.split() if word not in sw]
    return " ".join(txt)

df_train['text'] = df_train['text'].apply(preprocess_text)
df_val['text'] = df_val['text'].apply(preprocess_text)
df_test['text'] = df_test['text'].apply(preprocess_text)

In [ ]:
import numpy as np
import keras
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Activation, Input, Embedding, Conv1D, GlobalMaxPool1D, SimpleRNN, LSTM, GRU, Masking
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.callbacks import TensorBoard 
from keras.objectives import categorical_crossentropy
from keras.callbacks import EarlyStopping  

In [ ]:
text_corpus_train = df_train['text'].values
text_corpus_valid = df_val['text'].values
text_corpus_test = df_test['text'].values

In [ ]:
tokenizer = Tokenizer(num_words=None, 
                     filters='#$%&()*+-<=>@[\\]^_`{|}~\t\n',
                     lower = False, split = ' ')
tokenizer.fit_on_texts(text_corpus_train)

sequences_train = tokenizer.texts_to_sequences(text_corpus_train)
sequences_val = tokenizer.texts_to_sequences(text_corpus_valid)
sequences_test = tokenizer.texts_to_sequences(text_corpus_test)

word_count = len(tokenizer.index_word) + 1
training_length = max([len(i.split()) for i in text_corpus_train])

X_train = pad_sequences(sequences_train, maxlen=training_length)
X_valid = pad_sequences(sequences_val, maxlen=training_length)



In [ ]:
y_train = df_train['class'].values
y_val = df_val['class'].values

In [ ]:
model = Sequential()

model.add(
    Embedding(input_dim=word_count,
              input_length=training_length,
              output_dim=30,
              trainable=True,
              mask_zero=True))
model.add(Masking(mask_value=0.0))

model.add(SimpleRNN(64))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

model.compile(
    optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
early_stopping=EarlyStopping(monitor='val_loss')  


history = model.fit(X_train, y_train,
                    batch_size=512,
                    epochs=10,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[early_stopping])

Epoch 1/10
34/34 [==============================] - 5s 95ms/step - loss: 0.4181 - accuracy: 0.8598 - val_loss: 0.2171 - val_accuracy: 0.9435
Epoch 2/10
34/34 [==============================] - 3s 79ms/step - loss: 0.2698 - accuracy: 0.9298 - val_loss: 0.2158 - val_accuracy: 0.9435
Epoch 3/10
34/34 [==============================] - 3s 79ms/step - loss: 0.2452 - accuracy: 0.9302 - val_loss: 0.1269 - val_accuracy: 0.9549
Epoch 4/10
34/34 [==============================] - 3s 79ms/step - loss: 0.1401 - accuracy: 0.9507 - val_loss: 0.1373 - val_accuracy: 0.9554


In [ ]:
score = model.evaluate(X_valid, y_val, batch_size=512, verbose=1)
print('\n')
print('Test score:', score[0])
print('Test accuracy:', score[1])

5/5 [==============================] - 0s 21ms/step - loss: 0.1548 - accuracy: 0.9496


Test score: 0.15477579832077026
Test accuracy: 0.9495798349380493


# Какие проблемы у рекурентных сетей?

- затухают градиенты
- медленно, нужно всегда дойти до конца

Как решить? -> LSTM


<img src="images/lstm.png">


https://colah.github.io/posts/2015-08-Understanding-LSTMs/


Давайте, кратко посмотрим как это работает:


<img src="images/LSTMMaths.png">

In [ ]:
model = Sequential()

model.add(
    Embedding(input_dim=word_count,
              input_length=training_length,
              output_dim=30,
              trainable=True,
              mask_zero=True))
model.add(Masking(mask_value=0.0))
model.add(LSTM(64, recurrent_dropout=0.2))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

model.compile(
    optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

early_stopping=EarlyStopping(monitor='val_loss')  


history = model.fit(X_train, y_train,
                    batch_size=512,
                    epochs=10,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[early_stopping])

Epoch 1/10
34/34 [==============================] - 11s 241ms/step - loss: 0.5390 - accuracy: 0.9008 - val_loss: 0.2359 - val_accuracy: 0.9435
Epoch 2/10
34/34 [==============================] - 8s 229ms/step - loss: 0.2711 - accuracy: 0.9266 - val_loss: 0.2126 - val_accuracy: 0.9435
Epoch 3/10
34/34 [==============================] - 8s 226ms/step - loss: 0.2499 - accuracy: 0.9281 - val_loss: 0.1664 - val_accuracy: 0.9435
Epoch 4/10
34/34 [==============================] - 8s 227ms/step - loss: 0.1566 - accuracy: 0.9364 - val_loss: 0.0990 - val_accuracy: 0.9642
Epoch 5/10
34/34 [==============================] - 8s 226ms/step - loss: 0.0618 - accuracy: 0.9806 - val_loss: 0.1013 - val_accuracy: 0.9642


In [ ]:
score = model.evaluate(X_valid, y_val, batch_size=512, verbose=1)
print('\n')
print('Test score:', score[0])
print('Test accuracy:', score[1])

5/5 [==============================] - 0s 37ms/step - loss: 0.1351 - accuracy: 0.9603


Test score: 0.13506671786308289
Test accuracy: 0.9603174328804016


# Какие проблемы:

- вычислительно сложно -> медленнее
- на очень длинных последовательностях все равно затухает градиент


Зачем платить больше - уберем некоторые врата (точнее совместим) -> ускоримся, уменьшим число параметров -> GRU


<img src="images/gru.png">


GRU Math


<img src="images/GRUMath.png">


In [ ]:
model = Sequential()

model.add(
    Embedding(input_dim=word_count,
              input_length=training_length,
              output_dim=30,
              trainable=True,
              mask_zero=True))
model.add(Masking(mask_value=0.0))
model.add(GRU(64, recurrent_dropout=0.2))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

model.compile(
    optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

early_stopping=EarlyStopping(monitor='val_loss')  


history = model.fit(X_train, y_train,
                    batch_size=512,
                    epochs=10,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[early_stopping])

Epoch 1/10
34/34 [==============================] - 10s 203ms/step - loss: 0.5846 - accuracy: 0.8640 - val_loss: 0.2080 - val_accuracy: 0.9435
Epoch 2/10
34/34 [==============================] - 6s 188ms/step - loss: 0.2359 - accuracy: 0.9308 - val_loss: 0.1652 - val_accuracy: 0.9471
Epoch 3/10
34/34 [==============================] - 6s 190ms/step - loss: 0.1605 - accuracy: 0.9368 - val_loss: 0.1098 - val_accuracy: 0.9601
Epoch 4/10
34/34 [==============================] - 6s 189ms/step - loss: 0.0634 - accuracy: 0.9776 - val_loss: 0.1072 - val_accuracy: 0.9606
Epoch 5/10
34/34 [==============================] - 6s 190ms/step - loss: 0.0249 - accuracy: 0.9924 - val_loss: 0.1238 - val_accuracy: 0.9637


In [ ]:
score = model.evaluate(X_valid, y_val, batch_size=512, verbose=1)
print('\n')
print('Test score:', score[0])
print('Test accuracy:', score[1])

5/5 [==============================] - 0s 34ms/step - loss: 0.1687 - accuracy: 0.9566


Test score: 0.1686669886112213
Test accuracy: 0.9565826058387756


In [ ]:
train_corpus = " ".join(df_train["text"])
train_corpus = train_corpus.lower()

In [ ]:
import nltk
from nltk.tokenize import word_tokenize
nltk.download("punkt")

tokens = word_tokenize(train_corpus)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
max_words = 1000

In [ ]:
from nltk.probability import FreqDist
dist = FreqDist(tokens_filtered)
tokens_filtered_top = [pair[0] for pair in dist.most_common(max_words-1)]

In [ ]:
vocabulary = {v: k for k, v in dict(enumerate(tokens_filtered_top, 1)).items()}

In [ ]:
import numpy as np

def text_to_sequence(text, maxlen):
    result = []
    tokens = word_tokenize(text.lower())
    tokens_filtered = [word for word in tokens if word.isalnum()]
    for word in tokens_filtered:
        if word in vocabulary:
            result.append(vocabulary[word])
    padding = [0]*(maxlen-len(result))
    return padding + result[-maxlen:]

In [ ]:
max_len = 50

In [ ]:
x_train = np.asarray([text_to_sequence(text, max_len) for text in df_train["text"]], dtype=np.int32)
x_test = np.asarray([text_to_sequence(text, max_len) for text in df_test["text"]], dtype=np.int32)
x_val = np.asarray([text_to_sequence(text, max_len) for text in df_val["text"]], dtype=np.int32)

In [ ]:
import numpy as np
import keras
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Activation, Input, Embedding, Conv1D, GlobalMaxPool1D
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.callbacks import TensorBoard 
from keras.objectives import categorical_crossentropy
from keras.callbacks import EarlyStopping  

In [ ]:
num_classes = 2
y_train = keras.utils.to_categorical(df_train['class'], num_classes)
y_val = keras.utils.to_categorical(df_val['class'], num_classes)

In [ ]:
model = Sequential()
model.add(Embedding(input_dim=max_words, output_dim=256, input_length=max_len))
model.add(Conv1D(256, 3))
model.add(Conv1D(128, 3))
model.add(Activation("relu"))
model.add(GlobalMaxPool1D())
model.add(Dense(10))
model.add(Activation("relu"))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

In [ ]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [ ]:
batch_size = 256
epochs = 10

In [ ]:
tensorboard=TensorBoard(log_dir='./logs', write_graph=True, write_images=True)
early_stopping=EarlyStopping(monitor='val_loss')  


history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[tensorboard, early_stopping])

Epoch 1/10
68/68 [==============================] - 43s 624ms/step - loss: 0.3086 - accuracy: 0.9188 - val_loss: 0.1278 - val_accuracy: 0.9544
Epoch 2/10
68/68 [==============================] - 42s 620ms/step - loss: 0.1514 - accuracy: 0.9451 - val_loss: 0.1195 - val_accuracy: 0.9570
Epoch 3/10
68/68 [==============================] - 43s 626ms/step - loss: 0.1125 - accuracy: 0.9578 - val_loss: 0.1296 - val_accuracy: 0.9497


In [ ]:
score = model.evaluate(x_val, y_val, batch_size=batch_size, verbose=1)
print('\n')
print('Test score:', score[0])
print('Test accuracy:', score[1])

9/9 [==============================] - 1s 117ms/step - loss: 0.1599 - accuracy: 0.9435


Test score: 0.1599329710006714
Test accuracy: 0.9435107111930847


# Выводы

На текущих данных получается, что свёрточная не хуже рекурретнтной. 
Я использовал данные по твитам из первой домашки. Эти данные нек загрузились.

Думаю, что детальная настройка параметров позволит в итоге добиться лучших результатов, однако это требует даже  на таком датасете времени.

Кстати, почему мы не переносим вычисления на gpu?